In [ ]:
#Import packages

import json
import arcgis
import arcpy
import pandas as pd
import xml.etree.ElementTree as ET
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    value.__cause__ = None  # suppress chained exceptions
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback


In [ ]:
# Setting Input Excel

field_order_excel = input("Excel file path having ordered fields: \nEx: C:\\temp\\FCToBeFixed.xlsx (Without quotes)\n\n")

print("\nThis excel to be used for setting field order in ArcFM properties.")


In [ ]:
# Verifying input excel

sheet= pd.read_excel(field_order_excel, dtype=str)

full_table = pd.DataFrame(sheet,columns=['Feature Class','Field'])

if not(('Feature Class' in full_table.columns) and ('Field' in full_table.columns)):
    raise ProcessLookupError("\nInput excel does not have all 2 required columns. \nPlease provide valid input excel specified by the process.")
else:
    print("Great, It's a valid excel format.")

In [ ]:
# Read Excel file 

field_order = full_table.groupby('Feature Class')['Field'].apply(list).to_dict()

print(json.dumps(field_order, indent=1))
#print(field_order.keys())

In [ ]:
# Specify Exported ArcFM Properties XML for reference

Export_ArcFM_Properties_XML = input("Exported ArcFM Properties XML file path: \nEx: C:\\temp\\ArcFM_all_default_properties.xml(Without quotes)\n\n") 

Import_ArcFM_Properties_XML = Export_ArcFM_Properties_XML.replace('.xml','_ForImport.xml')

print("\nExported ArcFM Properties XML(%s) to be used for reference in setting field order according to excel(%s). \n\nA new XML file will be genrated for importing at this location \n%s" %(Export_ArcFM_Properties_XML,field_order_excel,Import_ArcFM_Properties_XML))


In [ ]:
# Preparing XML for import

tree = ET.parse(Export_ArcFM_Properties_XML)
root = tree.getroot()

for feature, fieldlist in field_order.items():
    feature_class_element = root.find('.//*[FEATURENAME="%s"]' %feature)
    if feature_class_element is None:
        raise ProcessLookupError("\n%s feature (specified in excel) does not exist in ArcFM Export XML. \nAborting process" %feature)
        
    subtype_elements = feature_class_element.findall('.//SUBTYPE')
    if len(subtype_elements) == 0:
        raise ProcessLookupError("\nSubType does not exist for %s feature in ArcFM Export XML. \nAborting process" %(feature))
    
    print("Applying field order for %s feature." %(feature))
    
    for subtype_element in subtype_elements:
        parent_map = {c: p for p in subtype_element.getiterator() for c in p}    
        child = subtype_element.find('.//QAINITIALIZED')
        index = list(parent_map[child]).index(child) + 1        
        for (display_index, field) in enumerate(fieldlist, start=index):
            field_element = subtype_element.find('.//*[FIELDNAME="%s"]' %field)
            
            if (field_element != None):
                subtype_element.remove(field_element)
                subtype_element.insert(display_index,field_element)
                #print(display_index, field,field_element)
            else:
                print("Can not set display order for %s field, it does not found in %s feature." %(field,feature))

tree.write(Import_ArcFM_Properties_XML)

print("\nA new XML file genrated for importing at this location \n%s" %(Import_ArcFM_Properties_XML))

    